## Using pyton to run SQL query

### SQL Tets

In [ ]:
# Importing library needed.

In [ ]:
import pandas as pd

In [ ]:
from sqlalchemy import create_engine, Table, Column, MetaData

In [ ]:
# Code to actiavte a temporary SQLite database.

In [ ]:
engine = create_engine('sqlite:///:memory:')

In [ ]:
# File path to open it in the enviroment.

In [ ]:
file_path = '/Users/jacobosimkin/Desktop/SQL Tables for DATA ANALYST.xlsx'

In [ ]:
xls = pd.ExcelFile(file_path)

In [ ]:
# Read the multple sheets on the file

In [ ]:
df_dept = pd.read_excel(file_path, sheet_name='DEPT')
df_doc_dept = pd.read_excel(file_path, sheet_name='DOC DEPT')
df_pat = pd.read_excel(file_path, sheet_name='Pat')
df_provider = pd.read_excel(file_path, sheet_name='Provider')
df_visit = pd.read_excel(file_path, sheet_name='Visit')

In [ ]:
#Code to store pandas DataFrames into SQLite database tables using SQLAlchemy.

In [ ]:
df_pat.to_sql('df_pat', engine, index=False)
df_visit.to_sql('df_visit', engine, index=False)
df_provider.to_sql('df_provider', engine, index=False)
df_dept.to_sql('df_dept', engine, index=False)
df_doc_dept.to_sql('df_doc_dept', engine, index=False)


In [ ]:
# SQL query.

In [ ]:
sql_query = """
SELECT
    P.PAT_ID,
    P.PATIENT,
    P.DOB,
    PD_CUR.DOC_NAME AS CUR_DOC,
    V.VISIT_DATE,
    V.VISIT_ID,
    PD_VISIT.DOC_NAME AS VISIT_DOC,
    D.DEPT_NAME,
    CASE WHEN D.FPG_DEPT_FLAG = 1 THEN 'Y' ELSE 'N' END AS VISIT_AT_FPG,
    CASE WHEN DD.FPG_DEPT_FLAG = 1 THEN 'Y' ELSE 'N' END AS CUR_DOC_FPG,
    CASE WHEN P.CUR_DOC_ID = V.VISIT_DOC_ID THEN 'Y' ELSE 'N' END AS CUR_DOC_IS_VIS_DOC,
    CAST((julianday(V.VISIT_DATE) - julianday(P.DOB)) / 365.25 AS INTEGER) AS Age_at_Visit
FROM
    df_pat P
JOIN
    df_visit V ON P.PAT_ID = V.PAT_ID
JOIN
    df_provider PD_CUR ON P.CUR_DOC_ID = PD_CUR.DOC_ID
JOIN
    df_provider PD_VISIT ON V.VISIT_DOC_ID = PD_VISIT.DOC_ID
JOIN
    df_dept D ON V.DEPT_ID = D.DEPT_ID
JOIN
    df_doc_dept DD ON P.CUR_DOC_ID = DD.DOC_ID AND DD.LINE = 1
WHERE
    D.FPG_DEPT_FLAG = 1;
"""

In [ ]:
# Pandas is used to execute a SQL query on a database via an SQLAlchemy Engine object (engine) and store the result in a Dataframe. 

In [ ]:
df_output = pd.read_sql_query(sql_query, engine)

In [ ]:
# Output

In [ ]:
df_output